In [1]:
import pandas as pd 
import tqdm
import multiprocessing

import pytorch_lightning as pl
import torch as th
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

import transformers

from sklearn.model_selection import train_test_split

In [2]:
th.cuda.get_device_name(0)
th.cuda.is_available()

True

In [3]:
model_name = 'bert-base-uncased'
batch_size = 32
lr = 1e-2
momentum = 0.9

In [4]:
df = pd.read_csv('../data/train.csv', index_col='id')

In [5]:
df.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [6]:
df.dropna(inplace=True)

In [7]:
df.count()

qid1            404287
qid2            404287
question1       404287
question2       404287
is_duplicate    404287
dtype: int64

In [8]:
# tokenizer = transformers.BertTokenizer.from_pretrained(model)
# tokenize = lambda x: tokenizer.tokenize(x)

In [9]:
# df['q1_len'] = df['question1'].apply(lambda x: len(tokenize(x)))
# df['q2_len'] = df['question2'].apply(lambda x: len(tokenize(x)))

In [10]:
# df['join_length'] = df['q1_len'] + df['q2_len']

In [11]:
# max_length = df['join_length'].max()

In [10]:
df = df.sample(frac=0.1, replace=False, random_state=1)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df[['question1', 'question2']], 
                                                    df['is_duplicate'],
                                                    test_size=0.2,
                                                   random_state=1)

In [16]:
class QuoraQuestionSimilarity(pl.LightningModule):
    def __init__(self, X_train, X_test, y_train, y_test):
        super().__init__()
        self.model = transformers.BertForSequenceClassification.from_pretrained(model_name)
        self.loss = th.nn.CrossEntropyLoss(reduction='none')
        self.x_train = X_train
        self.y_train = y_train
        self.x_test = X_test
        self.y_test = y_test
    
    def prepare_data(self):
        tokenizer = tokenizer = transformers.BertTokenizer.from_pretrained(model_name, num_labels=2)
        
        def _tokenize(x):
            return tokenizer.encode_plus(x['question1'], x['question2'], 
                                         max_length=330, pad_to_max_length=True,
                                         return_attention_mask=True, return_tensors='pt', 
                                         truncation=True)
        
        def _prepare(x, y):
            input_ids = []
            attention_masks = []
            token_type_ids = []
            for _, row in x.iterrows():
                encoded_data = _tokenize(row)
                input_ids.append(encoded_data['input_ids'])
                token_type_ids.append(encoded_data["token_type_ids"])
                attention_masks.append(encoded_data['attention_mask'])
                
            input_ids = th.cat(input_ids, dim=0)
            token_type_ids = th.cat(token_type_ids, dim=0)
            attention_masks = th.cat(attention_masks, dim=0)
            labels = th.tensor(y.values)
            return TensorDataset(input_ids, attention_masks, token_type_ids, labels)
        
        self.train_ds = _prepare(self.x_train, self.y_train)
        self.test_ds = _prepare(self.x_test, self.y_test)
    
    def train_dataloader(self):
        return DataLoader(self.train_ds, sampler = RandomSampler(self.train_ds), 
                          batch_size = batch_size, pin_memory=True, 
                          num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(self.test_ds, batch_size=batch_size, 
                          sampler = RandomSampler(self.test_ds), 
                          pin_memory=True, num_workers=4)  
    
    def configure_optimizers(self):
        return th.optim.SGD(
            self.parameters(),
            lr=lr,
            momentum=momentum
        )
    
    def training_step(self, batch, batch_idx):
        input_ids, attention_masks, token_type_ids, labels = batch
        logits = self.forward(input_ids, token_type_ids, attention_masks, labels)
        loss = self.loss(logits, labels).mean()
        return {'loss': loss, 'log': {'train_loss': loss}}

    def validation_step(self, batch, batch_idx):
        input_ids, attention_masks, token_type_ids, labels = batch
        logits = self.forward(input_ids, token_type_ids, attention_masks, labels)
        loss = self.loss(logits, labels)
        acc = (logits.argmax(-1) == labels).float()
        return {'loss': loss, 'acc': acc}

    def validation_epoch_end(self, outputs):
        loss = th.cat([o['loss'] for o in outputs], 0).mean()
        acc = th.cat([o['acc'] for o in outputs], 0).mean()
        out = {'val_loss': loss, 'val_acc': acc}
        print(out)
        return {**out, 'log':out}

    
    def forward(self, input_ids, token_type_ids, attention_masks, labels):
        loss, logits = self.model(input_ids, token_type_ids=token_type_ids, 
                             attention_mask=attention_masks, 
                             labels=labels)
        return logits
    

In [17]:
# (1 if th.cuda.is_available() else 0)

model = QuoraQuestionSimilarity(X_train[:5], X_test[:5], y_train[:5], y_test[:5])
trainer = pl.Trainer(
    default_root_dir='logs',
#     gpus=(1 if th.cuda.is_available() else 0),
    gpus=0,
    max_epochs=5,
    logger=TensorBoardLogger('logs/', name='qqs', version=0)
)
trainer.fit(model)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 109 M 
1 | loss  | CrossEntropyLoss              | 0     
/home/moshe/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


{'val_loss': tensor(0.5278), 'val_acc': tensor(0.8000)}


{'val_loss': tensor(4.6934), 'val_acc': tensor(0.2000)}


{'val_loss': tensor(7.3063), 'val_acc': tensor(0.2000)}


{'val_loss': tensor(7.4162), 'val_acc': tensor(0.2000)}


{'val_loss': tensor(5.2698), 'val_acc': tensor(0.2000)}


{'val_loss': tensor(1.7018), 'val_acc': tensor(0.2000)}



1